## Importing necessary libraries

In [ ]:
import flowermd
import gsd
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import numpy as np
import warnings
from cmeutils.visualize import FresnelGSD
from flowermd.base import Pack, Simulation, System, Molecule
from flowermd.base.forcefield import BaseHOOMDForcefield
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.library.forcefields import BeadSpring
from flowermd.utils import get_target_box_number_density
from mbuild.compound import Compound
from mbuild.lattice import Lattice
import unyt as u
warnings.filterwarnings('ignore')

## Switch to .CPU() if running on CPU, GPU if on GPU

In [ ]:
cpu = hoomd.device.GPU()

## Class defining flake geometry

In [ ]:
class Graphene(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        ) 
        Flakium = Compound(name="F", element="F") # defines a carbon atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined carbon atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated carbon lattice layers to the 'surface' compound, which represents our graphene structure 
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Graphene, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]
#OK, so we want to initialize a system with some chains and some flakes

## Forcefield parameters defining WCA interactions, essentially zero attraction, pure repulsion

In [ ]:
ff = BeadSpring(
    r_cut=2**(1/6),  
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000),
        "A-A": dict(r0=1.0, k=1000.0),  # increased k to avoid chain collapse
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   # moderate stiffness for chains
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), #need to turn this on later, messed up with straight chains
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)

## Simulation parameters

In [ ]:
N_chains = 100
initial_dens = 0.001
final_dens = 0.3
N_flakes = 10
chain_length = 10
dt = 0.005
temp = 3

## Initializing chains, flakes, and system parameters (like initial density -> final density)

In [ ]:
kg_chain = LJChain(lengths=chain_length,num_mols=N_chains)
sheet = Graphene(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False))
system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=N_flakes), kg_chain], 
              density=initial_dens, packing_expand_factor = 6, seed=2)
target_box = get_target_box_number_density(density=final_dens*u.Unit("nm**-3"),n_beads=(500+(N_chains*10)))

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Output files

In [ ]:
gsd = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.gsd"
log = f"{N_chains}_{chain_length}mer{N_flakes}f_{dt}dt.txt"

## Initializing simulation, shrinking, running, then flushing data into output files.

In [ ]:
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=cpu, dt = dt, gsd_write_freq=int(1000), log_file_name = log, gsd_file_name = gsd)
#sim.run_update_volume(final_box_lengths=[(1500/0.8)**1/3, (1500/0.8)**1/3, (1500/0.8)**1/3], kT=6.0, n_steps=5e6,tau_kt=100*sim.dt,period=10,thermalize_particles=True)
sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=5e6,tau_kt=100*sim.dt,period=10,thermalize_particles=True)
sim.run_NVT(n_steps=5e6, kT=temp, tau_kt=dt*100)
sim.flush_writers()

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 5000000; TPS: 816.72; ETA: 1.0 hours, 42.0 minutes
Step 2000 of 5000000; TPS: 1471.39; ETA: 56.6 minutes
Step 3000 of 5000000; TPS: 2006.72; ETA: 41.5 minutes
Step 4000 of 5000000; TPS: 2452.54; ETA: 34.0 minutes
Step 5000 of 5000000; TPS: 2789.05; ETA: 29.8 minutes
Step 6000 of 5000000; TPS: 3110.15; ETA: 26.8 minutes
Step 7000 of 5000000; TPS: 3375.26; ETA: 24.7 minutes
Step 8000 of 5000000; TPS: 3585.73; ETA: 23.2 minutes
Step 9000 of 5000000; TPS: 3759.25; ETA: 22.1 minutes
Step 10000 of 5000000; TPS: 3917.92; ETA: 21.2 minutes
Step 11000 of 5000000; TPS: 3094.42; ETA: 26.9 minutes
Step 12000 of 5000000; TPS: 2275.24; ETA: 36.5 minutes
Step 13000 of 5000000; TPS: 2397.2; ETA: 34.7 minutes
Step 14000 of 5000000; TPS: 2509.13; ETA: 33.1 minutes
Step 15000 of 5000000; TPS: 2608.46; ETA: 31.9 minutes
Step 16000 of 5000000; TPS: 2705.51; ETA: 30.7 minutes
Step 17000 of 5000000; TPS: 2797.66; ETA: 29.7 minutes
Step 18000 

RuntimeError: Particle with unique tag 633 is no longer in the simulation box.

Cartesian coordinates: 
x: 124.845 y: 14.3699 z: -184.722
Fractional coordinates: 
f.x: 1.41177 f.y: 0.604946 f.z: -0.849057
Local box lo: (-68.4632, -68.4632, -68.4632)
          hi: (68.4632, 68.4632, 68.4632)


## Visualization of final frame. Can adjust to whatever frame if you'd like

In [ ]:
sim_visualizer = FresnelGSD(gsd_file=gsd, frame=-1, view_axis=(1, 1, 1))
sim_visualizer.view()